# Streamlit App UI Example

This notebook demonstrates an app interface that runs alongside the dashboard.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY"
)

from trulens_eval import Tru

tru = Tru()
tru.reset_database()
tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)


✅ Key OPENAI_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Deleted 5 rows.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.49:8501 .


In [2]:
def load_app():
    from langchain.llms import OpenAI
    from langchain.chains import ConversationChain
    from langchain.memory import ConversationSummaryBufferMemory

    llm = OpenAI(temperature=0.9, max_tokens=128)

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        k=4,
        max_token_limit=64,
        llm=llm,
    )

    # Conversational app puts it all together.
    app = ConversationChain(
        llm=llm,
        memory=memory
    )

    return app 

app = load_app()

truchain = tru.Chain(app,
    app_id='Chain1_ChatApplication',
    tags = "prototype",
    metadata=[1,2,3],
    initial_app_loader=load_app
)

Wrote loadable app to /Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/examples/experimental/Chain1_ChatApplication.json and /Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/examples/experimental/Chain1_ChatApplication.dill.


In [3]:
from trulens_eval.appui import AppUI

# Interesting Select.App queries to look at:
# - app.memory.chat_memory.messages[:].content
# - app.memory.moving_summary_buffer

# Interesting Select.Record queries to look at:
# - app.memory.save_context....
# 

# aui = AppUI(app=truchain)
# aui.d

In [4]:
from trulens_eval.schema import AppDefinition

apps = AppDefinition.get_loadable_apps()